In [34]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import tensorflow as tf

from time import time

from edward.models import (
    Dirichlet, Categorical, ParamMixture)

plt.style.use('ggplot')

In [35]:
def build_toy_dataset(D, V, K):
    """
    Builds a toy dataset of D documents.
    
    Args:
        D - Number of documents
        V - Size of vocabulary
        K - Number of topics
        
    Returns:
        w - List of 1D NumPy arrays containing
            word tokens for each document
        theta - 2D NumPy array containing document
                topic distributions
        beta - 2D NumPy array containing topic
               token distributions
    """
    # Draw number of tokens for each document
    N = np.random.randint(low=100,
                          high=500,
                          size=D)
    
    # Draw topic distributions for each document
    theta = np.random.dirichlet(alpha=np.ones(K) * 0.1, 
                                size=D)

    # Create topics (non-overlapping)
    beta_values = np.array([K * 1 / V] * int(V / K) + [0.0] * int(V - V / K))
    beta = np.zeros([K, V])
    for k in range(K):
        beta[k, :] = np.roll(beta_values, int(k * V / K))
    
    # Draw tokens for each document
    w, z = [0] * D, [0] * D
    for d in range(D):
        # Draw token topic assignments
        z[d] = np.array([np.random.choice(range(K), size=N[d], p=theta[d, :])])[0]
        # Draw tokens
        w[d] = np.zeros(N[d])
        for n in range(N[d]):
            w[d][n] = np.random.choice(range(V), size=1, p=beta[z[d][n], :]) 

    return w, theta, beta

In [36]:
# Get toy dataset
D = 10
V = 30
K = 3
w_train, theta_train, beta_train = build_toy_dataset(D, V, K)

In [37]:
# Convert documents to STAN example's vectorized form

N = 0 # Total number of words in corpus

for d, wt in enumerate(w_train):
    N += wt.shape[0]
    
    # w_train_reshape: Vector containing all word instances in corpus
    # doc_train: Vector containing word instance to document assignments
    if d == 0:
        w_train_reshape = wt
        doc_train = np.ones(wt.shape[0]) * d
        
    else:
        w_train_reshape = np.concatenate([w_train_reshape, wt])
        doc_train = np.concatenate([doc_train, np.ones(wt.shape[0]) * d])
        
w_train = w_train_reshape

In [38]:
# Print topic token distributions
beta_train

array([[ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.1,
         0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.1,  0.1,
         0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1]])

In [39]:
###############
# MODEL
###############

doc = tf.placeholder(tf.int32, N)

alpha = tf.ones(K) * 1 / K #Dirichlet param for theta
beta = tf.ones(V) * 1 / V #Dirichlet param for phi

theta = Dirichlet(concentration=alpha, 
                  sample_shape=D,
                  name='theta')

phi = Dirichlet(concentration=beta, 
                sample_shape=K,
                name='phi')

z = Categorical(logits=tf.gather(theta, doc),
                name='z')

w = Categorical(probs=tf.gather(phi, z),
                name='w')


In [64]:
####################
#INFERENCE
####################

qtheta_variables = tf.Variable(tf.ones([D, K]),
                               name='qtheta-variables')

qtheta = Dirichlet(concentration=qtheta_variables,
                   name='qtheta')


qphi_variables = tf.Variable(tf.ones([K, V]),
                             name='qphi-variables')

qphi = Dirichlet(concentration=qphi_variables,
                 name='qphi')


qz_variables = tf.Variable(tf.nn.softplus(tf.ones([N, K])), 
                           name='qz-variables')

qz = Categorical(logits=qz_variables,
                 name='qz')


qz_var_init = np.ones([N, K]) * 1 / K
inference = ed.KLqp({phi: qphi, theta: qtheta, z: qz}, 
                    data={w: w_train, doc: doc_train})
tf.global_variables_initializer().run()
# inference.run()

inference.initialize()
tf.global_variables_initializer().run()
for _ in range(inference.n_iter):
    qz_var_est = qz_variables.eval()
    qphi_var_est = qphi_variables.eval()
    qtheta_var_est = qtheta_variables.eval()
    
    print('\nit: {}, qz_var_est: {}\n'.format(_, qz_var_est))
    print('\nit: {}, qphi_var_est: {}\n'.format(_, qphi_var_est))
    print('\nit: {}, qtheta_var_est: {}\n'.format(_, qtheta_var_est))
    
    info_dict = inference.update()#feed_dict={qz_variables: qz_est})
    inference.print_progress(info_dict)

inference.finalize()


it: 0, qz_var_est: [[ 1.31326163  1.31326163  1.31326163]
 [ 1.31326163  1.31326163  1.31326163]
 [ 1.31326163  1.31326163  1.31326163]
 ..., 
 [ 1.31326163  1.31326163  1.31326163]
 [ 1.31326163  1.31326163  1.31326163]
 [ 1.31326163  1.31326163  1.31326163]]


it: 0, qphi_var_est: [[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]


it: 0, qtheta_var_est: [[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]

   1/1000 [  0%]                                ETA: 2481s | Loss: 11387.982
it: 1, qz_var_est: [[ 1.41326165  1.41326165  1.2132616 ]
 [ 1.41326165  1.41326165  

  10/1000 [  1%]                                ETA: 251s | Loss: 13272.573 
it: 10, qz_var_est: [[ 1.54062521  1.37534308  1.19526696]
 [ 1.15976477  1.66980278  1.38548565]
 [ 1.15794086  1.12054563  1.91756129]
 ..., 
 [ 1.97673893  0.85105926  1.42249954]
 [ 1.75760889  2.11923194  0.5879041 ]
 [ 0.88124037  1.68050456  1.6637553 ]]


it: 10, qphi_var_est: [[ 0.86299145  0.34778392  1.33765256  1.40739346  0.94810474  0.16977891
   1.23608971  1.40450013  0.63278717  1.14671218  0.48025209  0.66771829
   0.64020592  0.78239191  1.26714432  0.8225435   1.29551589  1.15463507
   0.53091633  1.12478244  0.2849668   1.5734508   0.73144639  0.82905638
   1.05105519  0.93760848  0.93582988  1.25792766  1.00122857  0.72857511]
 [ 1.49888194  0.26595286  0.94154441  0.54230452  0.28726003  1.20842099
   1.22224474  0.97848594  0.37321258  0.58109319  1.2274245   1.00470901
   1.30162311  0.70778251  1.50799191  0.7596032   1.14183748  1.01525319
   1.11755276  0.64474267  1.3063432   0.731

InvalidArgumentError: indices[0] = 3 is not in [0, 3)
	 [[Node: inference_24/sample_24/Gather_4 = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](inference_24/sample_24/qphi_22/sample/Reshape, inference_24/sample_24/qz_26/sample/Reshape_1)]]

Caused by op 'inference_24/sample_24/Gather_4', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/runpy.py", line 171, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/torod/python_envs/pp/lib/python3.4/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/torod/python_envs/pp/lib/python3.4/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/torod/python_envs/pp/lib/python3.4/site-packages/ipykerne